int형 자료들은 float형으로 변환

In [358]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from collections import Counter
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

plt.rc("font", family = "Malgun Gothic")
sns.set(font="Malgun Gothic", 
rc={"axes.unicode_minus":False}, style='white')

In [359]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.drop(['ID'], axis=1, inplace=True)
test.drop(['ID'], axis=1, inplace=True)

In [360]:
train

,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,12480000,36 months,6 years,RENT,72000000,18.90,15,부채 통합,0,0,0.0,0.0,0.0,C
1,14400000,60 months,10+ years,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060.0,0.0,0.0,B
2,12000000,36 months,5 years,MORTGAGE,96000000,8.60,14,부채 통합,0,928644,151944.0,0.0,0.0,A
3,14400000,36 months,8 years,MORTGAGE,132000000,15.09,15,부채 통합,0,325824,153108.0,0.0,0.0,C
4,18000000,60 months,Unknown,RENT,71736000,25.39,19,주요 구매,0,228540,148956.0,0.0,0.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96289,14400000,36 months,10+ years,MORTGAGE,210000000,9.33,33,신용 카드,0,974580,492168.0,0.0,0.0,C
96290,28800000,60 months,10+ years,MORTGAGE,132000000,5.16,25,주택 개선,0,583728,855084.0,0.0,0.0,E
96291,14400000,36 months,1 year,MORTGAGE,84000000,11.24,22,신용 카드,0,1489128,241236.0,0.0,0.0,A
96292,15600000,36 months,5 years,MORTGAGE,66330000,17.30,21,부채 통합,2,1378368,818076.0,0.0,0.0,D


In [361]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96294 entries, 0 to 96293
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   대출금액          96294 non-null  int64  
 1   대출기간          96294 non-null  object 
 2   근로기간          96294 non-null  object 
 3   주택소유상태        96294 non-null  object 
 4   연간소득          96294 non-null  int64  
 5   부채_대비_소득_비율   96294 non-null  float64
 6   총계좌수          96294 non-null  int64  
 7   대출목적          96294 non-null  object 
 8   최근_2년간_연체_횟수  96294 non-null  int64  
 9   총상환원금         96294 non-null  int64  
 10  총상환이자         96294 non-null  float64
 11  총연체금액         96294 non-null  float64
 12  연체계좌수         96294 non-null  float64
 13  대출등급          96294 non-null  object 
dtypes: float64(4), int64(5), object(5)
memory usage: 10.3+ MB


In [362]:
X = train.drop(['대출등급'], axis=1)
y = train['대출등급']

In [363]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [364]:
y_train.value_counts()

B    23010
C    22129
A    13501
D    10612
E     5906
F     1537
G      340
Name: 대출등급, dtype: int64

In [365]:
y_val.value_counts()

B    5807
C    5494
A    3271
D    2742
E    1448
F     417
G      80
Name: 대출등급, dtype: int64

In [366]:
oversampler = RandomOverSampler(sampling_strategy='auto', random_state=42)
X_train_over, y_train_over = oversampler.fit_resample(X_train, y_train)
print(f"Class distribution after oversampling: {Counter(y_train_over)}")

Class distribution after oversampling: Counter({'A': 23010, 'C': 23010, 'B': 23010, 'E': 23010, 'D': 23010, 'F': 23010, 'G': 23010})


In [367]:
X_train_over

,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수
0,28800000,36 months,10+ years,MORTGAGE,120000000,10.10,20,신용 카드,0,2914824,625428.0,0.0,0.0
1,2880000,36 months,2 years,RENT,16200000,33.78,16,신용 카드,0,65172,28428.0,0.0,0.0
2,14400000,36 months,5 years,RENT,240000000,3.76,37,부채 통합,0,338304,125664.0,0.0,0.0
3,7800000,36 months,10+ years,RENT,40149600,22.92,28,신용 카드,0,385116,97944.0,0.0,0.0
4,17760000,60 months,1 year,RENT,54000000,13.41,29,부채 통합,0,442560,407424.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
161065,19860000,60 months,8 years,RENT,55200000,31.12,24,부채 통합,0,155520,399888.0,0.0,0.0
161066,18120000,60 months,6 years,RENT,43200000,15.73,11,부채 통합,0,277944,843000.0,0.0,0.0
161067,15570000,60 months,1 year,OWN,36106800,21.70,27,부채 통합,0,520968,1314360.0,0.0,0.0
161068,15030000,60 months,3 years,RENT,47040000,16.93,6,소규모 사업,0,367272,995136.0,0.0,0.0


In [368]:
X_val

,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수
79749,14400000,60 months,8 years,MORTGAGE,72000000,33.74,18,신용 카드,0,530268,427224.0,0.0,0.0
61621,34830000,36 months,10+ years,OWN,79200000,9.67,41,기타,3,741732,475968.0,0.0,0.0
127,14400000,36 months,1 year,RENT,91200000,9.81,12,신용 카드,1,309456,176700.0,0.0,0.0
65562,21900000,36 months,< 1 year,MORTGAGE,132000000,17.72,38,부채 통합,0,982080,510948.0,0.0,0.0
26735,12000000,36 months,10+ years,MORTGAGE,102000000,13.64,30,소규모 사업,0,559536,226152.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50679,29040000,60 months,3 years,RENT,66000000,19.18,12,부채 통합,0,1268280,1549476.0,0.0,0.0
43079,5760000,36 months,10+ years,RENT,53697600,16.06,25,휴가,0,527880,250512.0,0.0,0.0
54846,15570000,60 months,< 1 year,RENT,134400000,4.82,15,신용 카드,0,357120,365328.0,0.0,0.0
88333,28800000,36 months,1 year,MORTGAGE,74400000,14.36,49,신용 카드,2,2110164,614856.0,0.0,0.0


In [369]:
cate_column = [var for var in X_train if X_train[var].dtype == 'O']
num_column = [var for var in X_train if X_train[var].dtype != 'O'] 

In [370]:
# int 타입 변수들을 모두 float 형태로 변환하는 함수 작성 
def type_transform(data_set):
    for var in num_column:
        if data_set[var].dtype == 'int64':
            data_set[var] = data_set[var].astype(float)

type_transform(X_train_over)
type_transform(X_val)

In [371]:
X_train_over

,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수
0,28800000.0,36 months,10+ years,MORTGAGE,120000000.0,10.10,20.0,신용 카드,0.0,2914824.0,625428.0,0.0,0.0
1,2880000.0,36 months,2 years,RENT,16200000.0,33.78,16.0,신용 카드,0.0,65172.0,28428.0,0.0,0.0
2,14400000.0,36 months,5 years,RENT,240000000.0,3.76,37.0,부채 통합,0.0,338304.0,125664.0,0.0,0.0
3,7800000.0,36 months,10+ years,RENT,40149600.0,22.92,28.0,신용 카드,0.0,385116.0,97944.0,0.0,0.0
4,17760000.0,60 months,1 year,RENT,54000000.0,13.41,29.0,부채 통합,0.0,442560.0,407424.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
161065,19860000.0,60 months,8 years,RENT,55200000.0,31.12,24.0,부채 통합,0.0,155520.0,399888.0,0.0,0.0
161066,18120000.0,60 months,6 years,RENT,43200000.0,15.73,11.0,부채 통합,0.0,277944.0,843000.0,0.0,0.0
161067,15570000.0,60 months,1 year,OWN,36106800.0,21.70,27.0,부채 통합,0.0,520968.0,1314360.0,0.0,0.0
161068,15030000.0,60 months,3 years,RENT,47040000.0,16.93,6.0,소규모 사업,0.0,367272.0,995136.0,0.0,0.0


In [372]:
X_val

,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수
79749,14400000.0,60 months,8 years,MORTGAGE,72000000.0,33.74,18.0,신용 카드,0.0,530268.0,427224.0,0.0,0.0
61621,34830000.0,36 months,10+ years,OWN,79200000.0,9.67,41.0,기타,3.0,741732.0,475968.0,0.0,0.0
127,14400000.0,36 months,1 year,RENT,91200000.0,9.81,12.0,신용 카드,1.0,309456.0,176700.0,0.0,0.0
65562,21900000.0,36 months,< 1 year,MORTGAGE,132000000.0,17.72,38.0,부채 통합,0.0,982080.0,510948.0,0.0,0.0
26735,12000000.0,36 months,10+ years,MORTGAGE,102000000.0,13.64,30.0,소규모 사업,0.0,559536.0,226152.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50679,29040000.0,60 months,3 years,RENT,66000000.0,19.18,12.0,부채 통합,0.0,1268280.0,1549476.0,0.0,0.0
43079,5760000.0,36 months,10+ years,RENT,53697600.0,16.06,25.0,휴가,0.0,527880.0,250512.0,0.0,0.0
54846,15570000.0,60 months,< 1 year,RENT,134400000.0,4.82,15.0,신용 카드,0.0,357120.0,365328.0,0.0,0.0
88333,28800000.0,36 months,1 year,MORTGAGE,74400000.0,14.36,49.0,신용 카드,2.0,2110164.0,614856.0,0.0,0.0


In [373]:
def replace_work_period(data_set):

    data_set.replace({'근로기간' : '<1 year'}, '< 1 year', inplace=True)
    data_set.replace({'근로기간' : '10+years'}, '10+ years', inplace=True)
    data_set.replace({'근로기간' : '1 years'}, '1 year', inplace=True)
    data_set.replace({'근로기간' : '3'}, '3 years', inplace=True)
    data_set.replace({'근로기간' : 'Unknown'}, np.NaN, inplace=True)
    data_set.replace({'주택소유상태': 'ANY'}, 'MORTGAGE', inplace=True)

replace_work_period(X_train_over)
replace_work_period(X_val)

In [374]:
for var in X_train_over[cate_column]:
    print(var, X_train_over[var].unique())

대출기간 [' 36 months' ' 60 months']
근로기간 ['10+ years' '2 years' '5 years' '1 year' '4 years' '7 years' '< 1 year'
 '3 years' '8 years' '9 years' '6 years' nan]
주택소유상태 ['MORTGAGE' 'RENT' 'OWN']
대출목적 ['신용 카드' '부채 통합' '주요 구매' '기타' '이사' '주택 개선' '휴가' '소규모 사업' '재생 에너지' '주택'
 '의료' '자동차']


In [375]:
for var in X_train_over[cate_column]:
    print(var, X_val[var].unique())

대출기간 [' 60 months' ' 36 months']
근로기간 ['8 years' '10+ years' '1 year' '< 1 year' '2 years' nan '7 years'
 '6 years' '4 years' '5 years' '3 years' '9 years']
주택소유상태 ['MORTGAGE' 'OWN' 'RENT']
대출목적 ['신용 카드' '기타' '부채 통합' '소규모 사업' '주요 구매' '주택 개선' '의료' '자동차' '휴가' '이사' '주택'
 '재생 에너지']


In [376]:
def simple_imputation(df, column_name):
    # 결측치가 있는 열을 선택
    missing_values = df[column_name].isnull()
    
    # 결측치가 아닌 값들의 확률 분포 계산
    non_missing_values = df.loc[~missing_values, column_name]
    probabilities = non_missing_values.value_counts(normalize=True)
    
    # 결측치 대체
    imputed_values = np.random.choice(probabilities.index, size=missing_values.sum(), p=probabilities.values)
    
    # 대체된 값으로 결측치 채우기
    df.loc[missing_values, column_name] = imputed_values
    
    return df


# 단순 확률 대치법 적용
column_to_impute = '근로기간'
simple_imputation(X_train_over, column_to_impute)
simple_imputation(X_val, column_to_impute)

,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수
79749,14400000.0,60 months,8 years,MORTGAGE,72000000.0,33.74,18.0,신용 카드,0.0,530268.0,427224.0,0.0,0.0
61621,34830000.0,36 months,10+ years,OWN,79200000.0,9.67,41.0,기타,3.0,741732.0,475968.0,0.0,0.0
127,14400000.0,36 months,1 year,RENT,91200000.0,9.81,12.0,신용 카드,1.0,309456.0,176700.0,0.0,0.0
65562,21900000.0,36 months,< 1 year,MORTGAGE,132000000.0,17.72,38.0,부채 통합,0.0,982080.0,510948.0,0.0,0.0
26735,12000000.0,36 months,10+ years,MORTGAGE,102000000.0,13.64,30.0,소규모 사업,0.0,559536.0,226152.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50679,29040000.0,60 months,3 years,RENT,66000000.0,19.18,12.0,부채 통합,0.0,1268280.0,1549476.0,0.0,0.0
43079,5760000.0,36 months,10+ years,RENT,53697600.0,16.06,25.0,휴가,0.0,527880.0,250512.0,0.0,0.0
54846,15570000.0,60 months,< 1 year,RENT,134400000.0,4.82,15.0,신용 카드,0.0,357120.0,365328.0,0.0,0.0
88333,28800000.0,36 months,1 year,MORTGAGE,74400000.0,14.36,49.0,신용 카드,2.0,2110164.0,614856.0,0.0,0.0


In [377]:
X_train_over.isnull().sum()

대출금액            0
대출기간            0
근로기간            0
주택소유상태          0
연간소득            0
부채_대비_소득_비율     0
총계좌수            0
대출목적            0
최근_2년간_연체_횟수    0
총상환원금           0
총상환이자           0
총연체금액           0
연체계좌수           0
dtype: int64

In [378]:
X_val.isnull().sum()

대출금액            0
대출기간            0
근로기간            0
주택소유상태          0
연간소득            0
부채_대비_소득_비율     0
총계좌수            0
대출목적            0
최근_2년간_연체_횟수    0
총상환원금           0
총상환이자           0
총연체금액           0
연체계좌수           0
dtype: int64

In [379]:
def loan_purpose(data_set):

    data_set.replace({'대출목적' : '주택 개선'}, '주택', inplace=True)
    data_set.replace({'대출목적' : '이사'}, '주택', inplace=True)
    data_set.replace({'대출목적' : '주요 구매'}, '생활비', inplace=True)
    data_set.replace({'대출목적' : '휴가'}, '생활비', inplace=True)
    data_set.replace({'대출목적':'의료'}, '생활비', inplace=True)
    data_set.replace({'대출목적':'자동차'}, '생활비', inplace=True)
    data_set.replace({'대출목적':'소규모 사업'}, '기타', inplace=True)
    data_set.replace({'대출목적':'재생 에너지'}, '기타', inplace=True)

loan_purpose(X_train_over)
loan_purpose(X_val)

In [380]:
X_train_over['대출목적'].value_counts()

부채 통합    97331
신용 카드    30147
기타       13128
주택       13127
생활비       7337
Name: 대출목적, dtype: int64

In [381]:
X_val['대출목적'].value_counts()

부채 통합    11018
신용 카드     4827
주택        1394
기타        1193
생활비        827
Name: 대출목적, dtype: int64

In [382]:
# 다중공선성 확인 1

from statsmodels.stats.outliers_influence import variance_inflation_factor

def calculate_vif(data):
    vif_data = pd.DataFrame()
    vif_data["Variable"] = data.columns
    vif_data["VIF"] = [variance_inflation_factor(data.values, i) for i in range(data.shape[1])]
    return vif_data

# 독립 변수만 선택
independent_vars = X_train_over[num_column]

# VIF 계산
vif_result = calculate_vif(independent_vars)
print(vif_result)

       Variable       VIF
0          대출금액  7.010456
1          연간소득  2.415142
2   부채_대비_소득_비율  1.494830
3          총계좌수  4.041487
4  최근_2년간_연체_횟수  1.198528
5         총상환원금  1.675430
6         총상환이자  3.154539
7         총연체금액  1.012782
8         연체계좌수  1.034171


In [383]:
def log_num(data_set):

    data_set['대출금액'] = np.log1p(data_set['대출금액'])
    data_set['연간소득'] = np.log1p(data_set['연간소득'])
    data_set['총상환원금'] = np.log1p(data_set['총상환원금'])
    data_set['총상환이자'] = np.log1p(data_set['총상환이자'])

log_num(X_train_over)
log_num(X_val)

In [384]:
X_train_over

,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수
0,17.175886,36 months,10+ years,MORTGAGE,18.603002,10.10,20.0,신용 카드,0.0,14.885320,13.346193,0.0,0.0
1,14.873301,36 months,2 years,RENT,16.600522,33.78,16.0,신용 카드,0.0,11.084801,10.255165,0.0,0.0
2,16.482739,36 months,5 years,RENT,19.296149,3.76,37.0,부채 통합,0.0,12.731703,11.741375,0.0,0.0
3,15.869634,36 months,10+ years,RENT,17.508123,22.92,28.0,신용 카드,0.0,12.861302,11.492161,0.0,0.0
4,16.692459,60 months,1 year,RENT,17.804495,13.41,29.0,부채 통합,0.0,13.000334,12.917612,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
161065,16.804218,60 months,8 years,RENT,17.826474,31.12,24.0,부채 통합,0.0,11.954536,12.898942,0.0,0.0
161066,16.712527,60 months,6 years,RENT,17.581351,15.73,11.0,부채 통합,0.0,12.535179,13.644723,0.0,0.0
161067,16.560857,60 months,1 year,OWN,17.401992,21.70,27.0,부채 통합,0.0,13.163446,14.088861,0.0,0.0
161068,16.525559,60 months,3 years,RENT,17.666509,16.93,6.0,기타,0.0,12.813861,13.810636,0.0,0.0


In [385]:
X_val

,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수
79749,16.482739,60 months,8 years,MORTGAGE,18.092177,33.74,18.0,신용 카드,0.0,13.181140,12.965066,0.0,0.0
61621,17.365990,36 months,10+ years,OWN,18.187487,9.67,41.0,기타,3.0,13.516745,13.073108,0.0,0.0
127,16.482739,36 months,1 year,RENT,18.328565,9.81,12.0,신용 카드,1.0,12.642574,12.082214,0.0,0.0
65562,16.901997,36 months,< 1 year,MORTGAGE,18.698312,17.72,38.0,부채 통합,0.0,13.797429,13.144025,0.0,0.0
26735,16.300417,36 months,10+ years,MORTGAGE,18.440483,13.64,30.0,기타,0.0,13.234865,12.328967,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50679,17.184185,60 months,3 years,RENT,18.005165,19.18,12.0,부채 통합,0.0,14.053173,14.253428,0.0,0.0
43079,15.566448,36 months,10+ years,RENT,17.798879,16.06,25.0,생활비,0.0,13.176626,12.431266,0.0,0.0
54846,16.560857,60 months,< 1 year,RENT,18.716331,4.82,15.0,신용 카드,0.0,12.785830,12.808554,0.0,0.0
88333,17.175886,36 months,1 year,MORTGAGE,18.124967,14.36,49.0,신용 카드,2.0,14.562277,13.329145,0.0,0.0


In [386]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

encoder = OneHotEncoder(drop='first', sparse=False)
X_train_over_encoded = encoder.fit_transform(X_train_over[cate_column])
X_val_encoded = encoder.fit_transform(X_val[cate_column])

c:\Users\pc\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\pc\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [404]:
X_train_over_df = pd.DataFrame(X_train_over_encoded, columns=encoder.get_feature_names_out(cate_column))
X_val_df = pd.DataFrame(X_val_encoded, columns=encoder.get_feature_names_out(cate_column))

In [405]:
X_train_over_df

,대출기간_ 60 months,근로기간_10+ years,근로기간_2 years,근로기간_3 years,근로기간_4 years,근로기간_5 years,근로기간_6 years,근로기간_7 years,근로기간_8 years,근로기간_9 years,근로기간_< 1 year,주택소유상태_OWN,주택소유상태_RENT,대출목적_부채 통합,대출목적_생활비,대출목적_신용 카드,대출목적_주택
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161065,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
161066,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
161067,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
161068,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [406]:
X_val_df

,대출기간_ 60 months,근로기간_10+ years,근로기간_2 years,근로기간_3 years,근로기간_4 years,근로기간_5 years,근로기간_6 years,근로기간_7 years,근로기간_8 years,근로기간_9 years,근로기간_< 1 year,주택소유상태_OWN,주택소유상태_RENT,대출목적_부채 통합,대출목적_생활비,대출목적_신용 카드,대출목적_주택
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19254,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
19255,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
19256,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
19257,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [412]:
X_train_over[num_column]

,대출금액,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수
0,17.175886,18.603002,10.10,20.0,0.0,14.885320,13.346193,0.0,0.0
1,14.873301,16.600522,33.78,16.0,0.0,11.084801,10.255165,0.0,0.0
2,16.482739,19.296149,3.76,37.0,0.0,12.731703,11.741375,0.0,0.0
3,15.869634,17.508123,22.92,28.0,0.0,12.861302,11.492161,0.0,0.0
4,16.692459,17.804495,13.41,29.0,0.0,13.000334,12.917612,0.0,0.0
...,...,...,...,...,...,...,...,...,...
161065,16.804218,17.826474,31.12,24.0,0.0,11.954536,12.898942,0.0,0.0
161066,16.712527,17.581351,15.73,11.0,0.0,12.535179,13.644723,0.0,0.0
161067,16.560857,17.401992,21.70,27.0,0.0,13.163446,14.088861,0.0,0.0
161068,16.525559,17.666509,16.93,6.0,0.0,12.813861,13.810636,0.0,0.0


In [415]:
X_val[num_column]

,대출금액,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수
79749,16.482739,18.092177,33.74,18.0,0.0,13.181140,12.965066,0.0,0.0
61621,17.365990,18.187487,9.67,41.0,3.0,13.516745,13.073108,0.0,0.0
127,16.482739,18.328565,9.81,12.0,1.0,12.642574,12.082214,0.0,0.0
65562,16.901997,18.698312,17.72,38.0,0.0,13.797429,13.144025,0.0,0.0
26735,16.300417,18.440483,13.64,30.0,0.0,13.234865,12.328967,0.0,0.0
...,...,...,...,...,...,...,...,...,...
50679,17.184185,18.005165,19.18,12.0,0.0,14.053173,14.253428,0.0,0.0
43079,15.566448,17.798879,16.06,25.0,0.0,13.176626,12.431266,0.0,0.0
54846,16.560857,18.716331,4.82,15.0,0.0,12.785830,12.808554,0.0,0.0
88333,17.175886,18.124967,14.36,49.0,2.0,14.562277,13.329145,0.0,0.0


In [417]:
X_train_over_last = pd.concat([X_train_over_df, X_train_over[num_column]], axis=1)

In [418]:
X_train_over_last

,대출기간_ 60 months,근로기간_10+ years,근로기간_2 years,근로기간_3 years,근로기간_4 years,근로기간_5 years,근로기간_6 years,근로기간_7 years,근로기간_8 years,근로기간_9 years,...,대출목적_주택,대출금액,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,17.175886,18.603002,10.10,20.0,0.0,14.885320,13.346193,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,14.873301,16.600522,33.78,16.0,0.0,11.084801,10.255165,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,16.482739,19.296149,3.76,37.0,0.0,12.731703,11.741375,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,15.869634,17.508123,22.92,28.0,0.0,12.861302,11.492161,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,16.692459,17.804495,13.41,29.0,0.0,13.000334,12.917612,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161065,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,16.804218,17.826474,31.12,24.0,0.0,11.954536,12.898942,0.0,0.0
161066,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,16.712527,17.581351,15.73,11.0,0.0,12.535179,13.644723,0.0,0.0
161067,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,16.560857,17.401992,21.70,27.0,0.0,13.163446,14.088861,0.0,0.0
161068,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,16.525559,17.666509,16.93,6.0,0.0,12.813861,13.810636,0.0,0.0


In [393]:
# 랜덤포레스트 모델

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_over_last, y_train_over)

y_pred = rf.predict(X_val_last)

accuracy_score_rf = accuracy_score(y_val, y_pred)
print("accuracy:",accuracy_score_rf)

ValueError: Input X contains NaN.
RandomForestClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values